# Pipeline

> Extract: Genres, Cast, Director, Keywords, Description

> Concatenate: Equally weighted/ extra weights to actors, directors(follow different methods)

> Vectortize top 5k-10k words using **BoW** or **TF-IDF**

> Use **cosine_similarity** to get the close relativeness of each movie with other

> Use **enumerate** function to bind index and probabilitier

> Get **cosine_similarity**, and use **enumerate** to write the **Recommend** Fucntion

# Data Preprocessing

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
movies = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Deep learning/NLP/Movie Recommendation/tmdb_5000_movies.csv')
credits = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Deep learning/NLP/Movie Recommendation/tmdb_5000_credits.csv')

In [3]:
movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [4]:
credits.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


**Merge Both Dataframes**

In [5]:
df = movies.merge(credits,on='title')

In [6]:
df.shape

(4809, 23)

In [7]:
df.sample(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
1353,36000000,"[{""id"": 53, ""name"": ""Thriller""}, {""id"": 878, ""...",NaN,782,"[{""id"": 856, ""name"": ""paraplegic""}, {""id"": 125...",en,Gattaca,Science fiction drama about a future society i...,70.398356,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,106.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,There is no gene for the human spirit.,Gattaca,7.5,1808,782,"[{""cast_id"": 7, ""character"": ""Vincent Freeman""...","[{""credit_id"": ""52fe4275c3a36847f8020563"", ""de..."
3360,0,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 27, ""name...",http://robzombie.com/movies/the-devils-rejects/,1696,"[{""id"": 625, ""name"": ""sadistic""}, {""id"": 627, ...",en,The Devil's Rejects,The sequel to House of 1000 Corpses – the Fire...,16.754452,"[{""name"": ""Lions Gate Films"", ""id"": 35}, {""nam...",...,107.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}, ...",Released,Death walks behind. Hell waits ahead.,The Devil's Rejects,6.6,322,1696,"[{""cast_id"": 15, ""character"": ""Captain Spauldi...","[{""credit_id"": ""52fe430dc3a36847f8036cdd"", ""de..."


**Remove unnecessary features**

In [8]:
df.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'movie_id', 'cast', 'crew'],
      dtype='object')

**Consider**
* genres
* id - for future use
* keywords
* title
* overview
* cast
* crew

In [9]:
df = df[['id','genres','keywords','title','overview','cast','crew']]

In [10]:
df.sample(5)

,id,genres,keywords,title,overview,cast,crew
590,9882,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 28, ""name...","[{""id"": 187, ""name"": ""islam""}, {""id"": 789, ""na...",The Siege,The secret US abduction of a suspected terrori...,"[{""cast_id"": 9, ""character"": ""Anthony 'Hub' Hu...","[{""credit_id"": ""52fe4540c3a36847f80c3af5"", ""de..."
2190,15644,"[{""id"": 28, ""name"": ""Action""}, {""id"": 18, ""nam...","[{""id"": 10183, ""name"": ""independent film""}]",Paparazzi,A rising Hollywood actor decides to take perso...,"[{""cast_id"": 1, ""character"": ""Bo Laramie"", ""cr...","[{""credit_id"": ""5678044292514179dd008c37"", ""de..."
209,49040,"[{""id"": 28, ""name"": ""Action""}, {""id"": 53, ""nam...","[{""id"": 782, ""name"": ""assassin""}, {""id"": 1994,...",The Bourne Legacy,"New CIA operative, Aaron Cross experiences lif...","[{""cast_id"": 5, ""character"": ""Aaron Cross"", ""c...","[{""credit_id"": ""52fe4782c3a36847f8139c3b"", ""de..."
4548,43933,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 53, ""name...","[{""id"": 1299, ""name"": ""monster""}, {""id"": 1949,...",Monsters,Six years ago NASA discovered the possibility ...,"[{""cast_id"": 3, ""character"": ""Samantha Wynden""...","[{""credit_id"": ""52fe466cc3a36847f80fe201"", ""de..."
2562,17127,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...","[{""id"": 4513, ""name"": ""blind date""}, {""id"": 96...",The Object of My Affection,A pregnant New York social worker begins to de...,"[{""cast_id"": 1, ""character"": ""Nina Borowski"", ...","[{""credit_id"": ""58fb95ac92514163ec010863"", ""de..."


In [11]:
df.isna().sum()

,0
id,0
genres,0
keywords,0
title,0
overview,3
cast,0
crew,0


In [12]:
df.dropna(inplace=True)

In [13]:
df.isna().sum()

,0
id,0
genres,0
keywords,0
title,0
overview,0
cast,0
crew,0


In [14]:
df.duplicated().sum()

np.int64(0)

**Use regex to extract features from genres, keywords, cast, crew**

**Genres**

In [15]:
for i in range(10):
  print(df['genres'][102+i])

[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 878, "name": "Science Fiction"}]
[{"id": 14, "name": "Fantasy"}, {"id": 12, "name": "Adventure"}, {"id": 28, "name": "Action"}, {"id": 35, "name": "Comedy"}, {"id": 18, "name": "Drama"}]
[{"id": 12, "name": "Adventure"}, {"id": 28, "name": "Action"}, {"id": 18, "name": "Drama"}, {"id": 53, "name": "Thriller"}]
[{"id": 14, "name": "Fantasy"}]
[{"id": 14, "name": "Fantasy"}, {"id": 12, "name": "Adventure"}, {"id": 16, "name": "Animation"}, {"id": 35, "name": "Comedy"}, {"id": 10751, "name": "Family"}]
[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}]
[{"id": 878, "name": "Science Fiction"}, {"id": 28, "name": "Action"}, {"id": 53, "name": "Thriller"}, {"id": 12, "name": "Adventure"}]
[{"id": 12, "name": "Adventure"}, {"id": 10751, "name": "Family"}, {"id": 14, "name": "Fantasy"}]
[{"id": 36, "name": "History"}, {"id": 10749, "name": "Romance"}, {"id": 10752, "name": "War"}]

In [16]:
import re
def extract_genres(text):
  genre_pattern = r'\"name\": \"([\w _]+)\"'
  return re.findall(genre_pattern,text)

In [17]:
extract_genres(df['genres'][10])

['Adventure', 'Fantasy', 'Action', 'Science Fiction']

In [18]:
df['genres'] = df['genres'].apply(extract_genres)
df.head()

,id,genres,keywords,title,overview,cast,crew
0,19995,"[Action, Adventure, Fantasy, Science Fiction]","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,"[Adventure, Fantasy, Action]","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,"[Action, Adventure, Crime]","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",Spectre,A cryptic message from Bond’s past sends him o...,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,"[Action, Crime, Drama, Thriller]","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",The Dark Knight Rises,Following the death of District Attorney Harve...,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,"[Action, Adventure, Science Fiction]","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",John Carter,"John Carter is a war-weary, former military ca...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


**Keywords**

In [19]:
for i in range(10):
  print(df['keywords'][124+i])

[{"id": 2011, "name": "queen"}, {"id": 4344, "name": "musical"}, {"id": 7376, "name": "princess"}, {"id": 10085, "name": "betrayal"}, {"id": 10115, "name": "snowman"}, {"id": 10336, "name": "animation"}, {"id": 10385, "name": "reindeer"}, {"id": 10541, "name": "curse"}, {"id": 10794, "name": "snow"}, {"id": 11173, "name": "troll"}, {"id": 155801, "name": "mountain climber"}, {"id": 179430, "name": "aftercreditsstinger"}, {"id": 187056, "name": "woman director"}, {"id": 209714, "name": "3d"}]
[{"id": 83, "name": "saving the world"}, {"id": 310, "name": "artificial intelligence"}, {"id": 312, "name": "man vs machine"}, {"id": 779, "name": "martial arts"}, {"id": 780, "name": "kung fu"}, {"id": 1001, "name": "underground world"}, {"id": 1566, "name": "dream"}, {"id": 1701, "name": "hero"}, {"id": 1721, "name": "fight"}, {"id": 2812, "name": "computer virus"}, {"id": 2849, "name": "key"}, {"id": 2964, "name": "future"}, {"id": 3217, "name": "plato"}, {"id": 3219, "name": "precognition"}, {

In [20]:
df['keywords'] = df['keywords'].apply(extract_genres)
df.head()

,id,genres,keywords,title,overview,cast,crew
0,19995,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,"[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...",Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...",Spectre,A cryptic message from Bond’s past sends him o...,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...",The Dark Knight Rises,Following the death of District Attorney Harve...,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,"[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...",John Carter,"John Carter is a war-weary, former military ca...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


**Cast (top 5 )**

In [21]:
for i in range(10):
  print(df['cast'][343+i])

[{"cast_id": 18, "character": "Woody (voice)", "credit_id": "52fe4284c3a36847f80250d1", "gender": 2, "id": 31, "name": "Tom Hanks", "order": 0}, {"cast_id": 28, "character": "Buzz Lightyear (voice)", "credit_id": "52fe4284c3a36847f80250f5", "gender": 2, "id": 12898, "name": "Tim Allen", "order": 1}, {"cast_id": 19, "character": "Jessie (voice)", "credit_id": "52fe4284c3a36847f80250d5", "gender": 1, "id": 3234, "name": "Joan Cusack", "order": 2}, {"cast_id": 20, "character": "Prospector (voice)", "credit_id": "52fe4284c3a36847f80250d9", "gender": 2, "id": 7090, "name": "Kelsey Grammer", "order": 3}, {"cast_id": 21, "character": "Mr. Potato Head (voice)", "credit_id": "52fe4284c3a36847f80250dd", "gender": 2, "id": 7167, "name": "Don Rickles", "order": 4}, {"cast_id": 22, "character": "Slinky Dog (voice)", "credit_id": "52fe4284c3a36847f80250e1", "gender": 2, "id": 12899, "name": "Jim Varney", "order": 5}, {"cast_id": 23, "character": "Rex (voice)", "credit_id": "52fe4284c3a36847f80250e5"

In [22]:
import re
def extract_cast(text):
  pattern = r'\"name\": \"([\w _]+)\"'
  return re.findall(pattern,text)[:5]

In [23]:
extract_cast(df['cast'][343])

['Tom Hanks', 'Tim Allen', 'Joan Cusack', 'Kelsey Grammer', 'Don Rickles']

In [24]:
df['cast'] = df['cast'].apply(extract_cast)
df.head()

,id,genres,keywords,title,overview,cast,crew
0,19995,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",Avatar,"In the 22nd century, a paraplegic Marine is di...","[Sam Worthington, Zoe Saldana, Sigourney Weave...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,"[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...",Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Johnny Depp, Orlando Bloom, Keira Knightley, ...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...",Spectre,A cryptic message from Bond’s past sends him o...,"[Daniel Craig, Christoph Waltz, Ralph Fiennes,...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...",The Dark Knight Rises,Following the death of District Attorney Harve...,"[Christian Bale, Michael Caine, Gary Oldman, A...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,"[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...",John Carter,"John Carter is a war-weary, former military ca...","[Taylor Kitsch, Lynn Collins, Samantha Morton,...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


**Director**

In [25]:
df['crew'][1]

'[{"credit_id": "52fe4232c3a36847f800b579", "department": "Camera", "gender": 2, "id": 120, "job": "Director of Photography", "name": "Dariusz Wolski"}, {"credit_id": "52fe4232c3a36847f800b4fd", "department": "Directing", "gender": 2, "id": 1704, "job": "Director", "name": "Gore Verbinski"}, {"credit_id": "52fe4232c3a36847f800b54f", "department": "Production", "gender": 2, "id": 770, "job": "Producer", "name": "Jerry Bruckheimer"}, {"credit_id": "52fe4232c3a36847f800b503", "department": "Writing", "gender": 2, "id": 1705, "job": "Screenplay", "name": "Ted Elliott"}, {"credit_id": "52fe4232c3a36847f800b509", "department": "Writing", "gender": 2, "id": 1706, "job": "Screenplay", "name": "Terry Rossio"}, {"credit_id": "52fe4232c3a36847f800b57f", "department": "Editing", "gender": 0, "id": 1721, "job": "Editor", "name": "Stephen E. Rivkin"}, {"credit_id": "52fe4232c3a36847f800b585", "department": "Editing", "gender": 2, "id": 1722, "job": "Editor", "name": "Craig Wood"}, {"credit_id": "52f

In [26]:
def extract_director(text):
  pattern = r'\"job\": \"Director\", \"name\": \"([\w ]+)\"'
  return re.findall(pattern,text)

In [27]:
extract_director(df['crew'][1461])

['Moustapha Akkad']

In [28]:
df['crew'] = df['crew'].apply(extract_director)
df.head()

,id,genres,keywords,title,overview,cast,crew
0,19995,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",Avatar,"In the 22nd century, a paraplegic Marine is di...","[Sam Worthington, Zoe Saldana, Sigourney Weave...",[James Cameron]
1,285,"[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...",Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Johnny Depp, Orlando Bloom, Keira Knightley, ...",[Gore Verbinski]
2,206647,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...",Spectre,A cryptic message from Bond’s past sends him o...,"[Daniel Craig, Christoph Waltz, Ralph Fiennes,...",[Sam Mendes]
3,49026,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...",The Dark Knight Rises,Following the death of District Attorney Harve...,"[Christian Bale, Michael Caine, Gary Oldman, A...",[Christopher Nolan]
4,49529,"[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...",John Carter,"John Carter is a war-weary, former military ca...","[Taylor Kitsch, Lynn Collins, Samantha Morton,...",[Andrew Stanton]


In [29]:
df['overview'] = df['overview'].apply(lambda x:x.split())
df.head()

,id,genres,keywords,title,overview,cast,crew
0,19995,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Sam Worthington, Zoe Saldana, Sigourney Weave...",[James Cameron]
1,285,"[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...",Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Johnny Depp, Orlando Bloom, Keira Knightley, ...",[Gore Verbinski]
2,206647,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...",Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Daniel Craig, Christoph Waltz, Ralph Fiennes,...",[Sam Mendes]
3,49026,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...",The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[Christian Bale, Michael Caine, Gary Oldman, A...",[Christopher Nolan]
4,49529,"[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...",John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[Taylor Kitsch, Lynn Collins, Samantha Morton,...",[Andrew Stanton]


In [30]:

df['genres'][10]

['Adventure', 'Fantasy', 'Action', 'Science Fiction']

In [31]:
for i in df['genres'][10]:
    i = re.sub(r'\s+',"",i)
    print(i)

Adventure
Fantasy
Action
ScienceFiction


**Removing space between words**

In [32]:
df['genres'] = df['genres'].apply(lambda x: [re.sub(r'\s+',"",i) for i in x])
df['keywords'] = df['keywords'].apply(lambda x: [re.sub(r'\s+',"",i) for i in x])
df['cast'] = df['cast'].apply(lambda x: [re.sub(r'\s+',"",i) for i in x])
df['crew'] = df['crew'].apply(lambda x: [re.sub(r'\s+',"",i) if x is not None else "" for i in x])        # Handle potential None values in crew

In [33]:
df.head(5)

,id,genres,keywords,title,overview,cast,crew
0,19995,"[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...",Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[SamWorthington, ZoeSaldana, SigourneyWeaver, ...",[JamesCameron]
1,285,"[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...",Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[JohnnyDepp, OrlandoBloom, KeiraKnightley, Bil...",[GoreVerbinski]
2,206647,"[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...",Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[DanielCraig, ChristophWaltz, RalphFiennes, Mo...",[SamMendes]
3,49026,"[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...",The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[ChristianBale, MichaelCaine, GaryOldman, Anne...",[ChristopherNolan]
4,49529,"[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...",John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[TaylorKitsch, LynnCollins, SamanthaMorton, Wi...",[AndrewStanton]


In [34]:
for i in range(10):
  print(df['overview'][i])
  print(len(df['overview'][i]))

['In', 'the', '22nd', 'century,', 'a', 'paraplegic', 'Marine', 'is', 'dispatched', 'to', 'the', 'moon', 'Pandora', 'on', 'a', 'unique', 'mission,', 'but', 'becomes', 'torn', 'between', 'following', 'orders', 'and', 'protecting', 'an', 'alien', 'civilization.']
28
['Captain', 'Barbossa,', 'long', 'believed', 'to', 'be', 'dead,', 'has', 'come', 'back', 'to', 'life', 'and', 'is', 'headed', 'to', 'the', 'edge', 'of', 'the', 'Earth', 'with', 'Will', 'Turner', 'and', 'Elizabeth', 'Swann.', 'But', 'nothing', 'is', 'quite', 'as', 'it', 'seems.']
34
['A', 'cryptic', 'message', 'from', 'Bond’s', 'past', 'sends', 'him', 'on', 'a', 'trail', 'to', 'uncover', 'a', 'sinister', 'organization.', 'While', 'M', 'battles', 'political', 'forces', 'to', 'keep', 'the', 'secret', 'service', 'alive,', 'Bond', 'peels', 'back', 'the', 'layers', 'of', 'deceit', 'to', 'reveal', 'the', 'terrible', 'truth', 'behind', 'SPECTRE.']
41
['Following', 'the', 'death', 'of', 'District', 'Attorney', 'Harvey', 'Dent,', 'Batma

In [35]:
df['tags1'] = df['cast'] + df['genres'] + df['crew'] + df['keywords'] + df['overview']
df.head()

,id,genres,keywords,title,overview,cast,crew,tags1
0,19995,"[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...",Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[SamWorthington, ZoeSaldana, SigourneyWeaver, ...",[JamesCameron],"[SamWorthington, ZoeSaldana, SigourneyWeaver, ..."
1,285,"[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...",Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[JohnnyDepp, OrlandoBloom, KeiraKnightley, Bil...",[GoreVerbinski],"[JohnnyDepp, OrlandoBloom, KeiraKnightley, Bil..."
2,206647,"[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...",Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[DanielCraig, ChristophWaltz, RalphFiennes, Mo...",[SamMendes],"[DanielCraig, ChristophWaltz, RalphFiennes, Mo..."
3,49026,"[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...",The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[ChristianBale, MichaelCaine, GaryOldman, Anne...",[ChristopherNolan],"[ChristianBale, MichaelCaine, GaryOldman, Anne..."
4,49529,"[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...",John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[TaylorKitsch, LynnCollins, SamanthaMorton, Wi...",[AndrewStanton],"[TaylorKitsch, LynnCollins, SamanthaMorton, Wi..."


In [36]:
new_df = df[['id','title','tags1']]
new_df.head()

,id,title,tags1
0,19995,Avatar,"[SamWorthington, ZoeSaldana, SigourneyWeaver, ..."
1,285,Pirates of the Caribbean: At World's End,"[JohnnyDepp, OrlandoBloom, KeiraKnightley, Bil..."
2,206647,Spectre,"[DanielCraig, ChristophWaltz, RalphFiennes, Mo..."
3,49026,The Dark Knight Rises,"[ChristianBale, MichaelCaine, GaryOldman, Anne..."
4,49529,John Carter,"[TaylorKitsch, LynnCollins, SamanthaMorton, Wi..."


**Lowercasing**

In [37]:
new_df['title'] = new_df['title'].apply(lambda x:x.lower())
new_df['tags1']  = new_df['tags1'].apply(lambda x:" ".join(x))
new_df['tags1'] = new_df['tags1'].apply(lambda x:x.lower())

/tmp/ipython-input-651841498.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['title'] = new_df['title'].apply(lambda x:x.lower())
/tmp/ipython-input-651841498.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags1']  = new_df['tags1'].apply(lambda x:" ".join(x))
/tmp/ipython-input-651841498.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https

In [38]:
new_df.head()

,id,title,tags1
0,19995,avatar,samworthington zoesaldana sigourneyweaver step...
1,285,pirates of the caribbean: at world's end,johnnydepp orlandobloom keiraknightley billnig...
2,206647,spectre,danielcraig christophwaltz ralphfiennes monica...
3,49026,the dark knight rises,christianbale michaelcaine garyoldman annehath...
4,49529,john carter,taylorkitsch lynncollins samanthamorton willem...


**Stemming**

In [39]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

def stemming(text):
  y = []
  for i in text.split():
    y.append(ps.stem(i))
  return " ".join(y)

In [40]:
print(new_df['tags1'][0])
print(stemming(new_df['tags1'][0]))

samworthington zoesaldana sigourneyweaver stephenlang michellerodriguez action adventure fantasy sciencefiction jamescameron cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d in the 22nd century, a paraplegic marine is dispatched to the moon pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.
samworthington zoesaldana sigourneyweav stephenlang michellerodriguez action adventur fantasi sciencefict jamescameron cultureclash futur spacewar spacecoloni societi spacetravel futurist romanc space alien tribe alienplanet cgi marin soldier battl loveaffair antiwar powerrel mindandsoul 3d in the 22nd century, a parapleg marin is dispatch to the moon pandora on a uniqu mission, but becom torn between follow order and protect an alien civilization.


In [41]:
new_df['tags1'] = new_df['tags1'].apply(stemming)
new_df.head()

/tmp/ipython-input-3514828447.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags1'] = new_df['tags1'].apply(stemming)


,id,title,tags1
0,19995,avatar,samworthington zoesaldana sigourneyweav stephe...
1,285,pirates of the caribbean: at world's end,johnnydepp orlandobloom keiraknightley billnig...
2,206647,spectre,danielcraig christophwaltz ralphfienn monicabe...
3,49026,the dark knight rises,christianbal michaelcain garyoldman annehathaw...
4,49529,john carter,taylorkitsch lynncollin samanthamorton willemd...


# Vectorization
with stopwords removal

In [42]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')
tfidf = TfidfVectorizer(max_features=5000,stop_words='english')

> Using BagofWords vectorization her

> Can use TF-IDF too

In [43]:
vectors = cv.fit_transform(new_df['tags1']).toarray()
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [44]:
print(vectors.shape)

(4806, 5000)


In [45]:
for i in cv.get_feature_names_out():
  print(i,end=" ")

000 007 10 100 11 12 13 14 15 16 17 17th 18 18th 18thcenturi 19 1910 1920 1930 1940 1944 1950 1950s 1960 1960s 1970 1970s 1980 1985 1990 19th 19thcenturi 20 200 2009 20th 24 25 30 300 3d 40 50 500 60 70 aaron aaroneckhart abandon abduct abigailbreslin abil abl aboard abov abus academi accept access accid accident acclaim accompani accomplish account accus ace achiev act action actionhero activ activist activities actor actress actual adam adambrodi adamsandl adamscott adamshankman adapt add addict adjust admir admit adolesc adopt ador adrienbrodi adult adulteri adulthood advanc adventur adventure advertis advic advis affair affect afghanistan africa african africanamerican aftercreditssting afterlif aftermath ag age agediffer agency agenda agent agents aggress ago agre ahead aid aidanquinn ail aim air airplan airplanecrash airport aka al alabama alan alanalda alanarkin alanrickman alaska albert albertbrook alcohol alecbaldwin alex alexapenavega alexisbledel alexkendrick alexpettyf alfr

# Prediction Function
**Using**
> cosine similarity

> enumerate function

**Using cosine similarity**

to get the closeness of the movies in terms of cosine angle/distance

In [73]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vectors)

In [72]:
similarity.shape

(4806, 4806)

In [48]:
similarity

array([[1.        , 0.0836242 , 0.08492078, ..., 0.04441156, 0.        ,
        0.        ],
       [0.0836242 , 1.        , 0.06154575, ..., 0.02414023, 0.        ,
        0.02654659],
       [0.08492078, 0.06154575, 1.        , ..., 0.02451452, 0.        ,
        0.        ],
       ...,
       [0.04441156, 0.02414023, 0.02451452, ..., 1.        , 0.0418121 ,
        0.04229549],
       [0.        , 0.        , 0.        , ..., 0.0418121 , 1.        ,
        0.0919601 ],
       [0.        , 0.02654659, 0.        , ..., 0.04229549, 0.0919601 ,
        1.        ]])

In [49]:
# similarity of 1st movie with all movies
print(similarity[0].shape)
print(similarity[0])

(4806,)
[1.         0.0836242  0.08492078 ... 0.04441156 0.         0.        ]


> **Writing a function that takes 1 movie name as input, and recommends 5 similar movies**

> **Use enumerate function to bind movie index to probabilities**



In [50]:
list(enumerate(similarity[0]))

[(0, np.float64(1.0000000000000002)),
 (1, np.float64(0.08362420100070908)),
 (2, np.float64(0.08492077756084468)),
 (3, np.float64(0.07206329937749)),
 (4, np.float64(0.18685673434682065)),
 (5, np.float64(0.10795837927188262)),
 (6, np.float64(0.03912553515233695)),
 (7, np.float64(0.1416576493949657)),
 (8, np.float64(0.05661385170722978)),
 (9, np.float64(0.09443843292997713)),
 (10, np.float64(0.09883324222148016)),
 (11, np.float64(0.09245003270420486)),
 (12, np.float64(0.08920515501750789)),
 (13, np.float64(0.04358136336404089)),
 (14, np.float64(0.12503908082008883)),
 (15, np.float64(0.06793662204867575)),
 (16, np.float64(0.07692307692307693)),
 (17, np.float64(0.13797289239745258)),
 (18, np.float64(0.0930725988812425)),
 (19, np.float64(0.08716272672808179)),
 (20, np.float64(0.05683630384789149)),
 (21, np.float64(0.10390486669322622)),
 (22, np.float64(0.06677805632941779)),
 (23, np.float64(0.08627959628145762)),
 (24, np.float64(0.053376051268362375)),
 (25, np.float6

In [51]:
sorted(list(enumerate(similarity[0])),reverse=True)

[(4805, np.float64(0.0)),
 (4804, np.float64(0.0)),
 (4803, np.float64(0.04441155916843275)),
 (4802, np.float64(0.04622501635210243)),
 (4801, np.float64(0.019003715589632167)),
 (4800, np.float64(0.0)),
 (4799, np.float64(0.05264981264926564)),
 (4798, np.float64(0.042051713353118)),
 (4797, np.float64(0.0)),
 (4796, np.float64(0.0)),
 (4795, np.float64(0.0)),
 (4794, np.float64(0.0)),
 (4793, np.float64(0.053376051268362375)),
 (4792, np.float64(0.0)),
 (4791, np.float64(0.0)),
 (4790, np.float64(0.05661385170722978)),
 (4789, np.float64(0.06100888760865632)),
 (4788, np.float64(0.0)),
 (4787, np.float64(0.01887128390240993)),
 (4786, np.float64(0.018248296715045298)),
 (4785, np.float64(0.019418390934515434)),
 (4784, np.float64(0.042796049251091296)),
 (4783, np.float64(0.0)),
 (4782, np.float64(0.025318484177091666)),
 (4781, np.float64(0.05751973085430508)),
 (4780, np.float64(0.0)),
 (4779, np.float64(0.0)),
 (4778, np.float64(0.0)),
 (4777, np.float64(0.11322770341445956)),
 (

In [52]:
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])

[(0, np.float64(1.0000000000000002)),
 (1214, np.float64(0.2850557338444825)),
 (507, np.float64(0.25231028011870793)),
 (3728, np.float64(0.2506402059138015)),
 (539, np.float64(0.24019223070763068)),
 (1202, np.float64(0.23870495801314426)),
 (2405, np.float64(0.2360960823249428)),
 (4046, np.float64(0.22489899383695006)),
 (778, np.float64(0.22422426466543746)),
 (61, np.float64(0.22417941532712204)),
 (1192, np.float64(0.21982600255746473)),
 (1916, np.float64(0.21977383072747692)),
 (973, np.float64(0.21182963643408081)),
 (2782, np.float64(0.21182963643408081)),
 (172, np.float64(0.20965696734438366)),
 (2329, np.float64(0.20587905489225486)),
 (151, np.float64(0.20483662259967564)),
 (582, np.float64(0.20380986614602725)),
 (322, np.float64(0.20254787341673336)),
 (3606, np.float64(0.20174251088960077)),
 (577, np.float64(0.20131905799006777)),
 (972, np.float64(0.20131905799006777)),
 (4190, np.float64(0.20033416898825335)),
 (1440, np.float64(0.20016019225635892)),
 (1087, np.

In [53]:
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])[1:6]

[(1214, np.float64(0.2850557338444825)),
 (507, np.float64(0.25231028011870793)),
 (3728, np.float64(0.2506402059138015)),
 (539, np.float64(0.24019223070763068)),
 (1202, np.float64(0.23870495801314426))]

**Recommender Function**

In [67]:
def recommend(movie):
  movie = movie.lower()
  movie_index = new_df[new_df['title']==movie].index[0]
  movies_list = sorted(list(enumerate(similarity[movie_index])),reverse=True,key=lambda x:x[1])[1:6]

  for i in movies_list:
    print(new_df.iloc[i[0]].title)

In [68]:
for i in range(10):
  print(df['title'][i],end=",")

Avatar,Pirates of the Caribbean: At World's End,Spectre,The Dark Knight Rises,John Carter,Spider-Man 3,Tangled,Avengers: Age of Ultron,Harry Potter and the Half-Blood Prince,Batman v Superman: Dawn of Justice,

# Evaluation

In [70]:
for i in range(10):
  movie = input("Enter a movie name:")
  recommend(movie)
  print('\n')

Enter a movie name:Avatar
aliens vs predator: requiem
independence day
falcon rising
titan a.e.
predators


Enter a movie name:Pirates of the Caribbean: At World's End
pirates of the caribbean: dead man's chest
pirates of the caribbean: the curse of the black pearl
pirates of the caribbean: on stranger tides
20,000 leagues under the sea
life of pi


Enter a movie name:Spectre
quantum of solace
skyfall
never say never again
from russia with love
octopussy


Enter a movie name:The Dark Knight Rises
the dark knight
batman begins
batman returns
batman
batman forever


Enter a movie name:John Carter
riddick
krrish
the other side of heaven
the legend of hercules
get carter


Enter a movie name:Spider-Man 3
spider-man
spider-man 2
the amazing spider-man 2
the amazing spider-man
arachnophobia


Enter a movie name:Avengers: Age of Ultron
the avengers
iron man 2
captain america: civil war
thor
the wolverine


Enter a movie name:Harry Potter and the Half-Blood Prince
harry potter and the order of